In [1]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

from ibge import *
from utils import *

In [2]:
# Initialize DuckDB connection
conn = duckdb.connect("piper.db")

In [3]:
result = conn.execute("SELECT COUNT(*) FROM gbif").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 44,800


In [15]:
df = pd.read_csv("species.csv", sep=";")
df.head()

,species
0,Piper andreanum C.DC.
1,Piper barbatum Kunth
2,Piper laguna-cochanum Trel. & Yunck.
3,Piper fortunaense Tebbs
4,Piper pilirameum C.DC.


In [38]:
conn.execute("SELECT COUNT(*) FROM gbif as g LIMIT 5").df()

,count_star()
0,44800


In [ ]:
species = tuple(df["species"].unique())
conn.execute(f"""
    SELECT g.scientificName, g.species \
        FROM gbif as g \
        WHERE g.scientificName in {species}
""").df()

,scientificName,species
0,Piper tuberculatum Jacq.,Piper tuberculatum
1,Piper aereum Trel.,Piper aereum
2,Piper auritum Kunth,Piper auritum
3,Piper aereum Trel.,Piper aereum
4,Piper melanocladum C.DC.,Piper melanocladum
...,...,...
9639,Piper hispidum Sw.,Piper hispidum
9640,Piper aduncum L.,Piper aduncum
9641,Piper cernuum Vell.,Piper cernuum
9642,Piper amalago L.,Piper amalago
